<a href="https://colab.research.google.com/github/Hemanthxin/DARVE/blob/main/DARVE_YOLOv8_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics==8.1.0 torch torchvision torchaudio
!pip install transformers timm scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.4 MB/s eta 0:00:00


In [2]:
!ls -lh /content/best.pt
!file /content/best.pt


-rw-r--r-- 1 root root 84M Jan 14 05:49 /content/best.pt
/content/best.pt: Zip archive data, at least v0.0 to extract, compression method=store


In [11]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2
!pip install ultralytics==8.1.0


Found existing installation: torch 2.9.0+cpu
Uninstalling torch-2.9.0+cpu:
  Successfully uninstalled torch-2.9.0+cpu
Found existing installation: torchvision 0.24.0+cpu
Uninstalling torchvision-0.24.0+cpu:
  Successfully uninstalled torchvision-0.24.0+cpu
Found existing installation: torchaudio 2.9.0+cpu
Uninstalling torchaudio-2.9.0+cpu:
  Successfully uninstalled torchaudio-2.9.0+cpu
ERROR: Could not find a version that satisfies the requirement torch==2.1.2 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 954.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import ultralytics.nn.tasks

torch.serialization.add_safe_globals([
    ultralytics.nn.tasks.DetectionModel
])


In [20]:
import torch
from ultralytics import YOLO
import ultralytics.nn.modules.conv
import ultralytics.nn.modules.block
import ultralytics.nn.modules.head

# Add necessary classes to safe globals for unpickling
torch.serialization.add_safe_globals([
    torch.nn.modules.container.Sequential,
    ultralytics.nn.modules.conv.Conv,
    torch.nn.modules.conv.Conv2d,
    torch.nn.modules.batchnorm.BatchNorm2d,
    torch.nn.modules.activation.SiLU,
    ultralytics.nn.modules.block.C2f,
    torch.nn.modules.container.ModuleList,
    ultralytics.nn.modules.block.Bottleneck,
    ultralytics.nn.modules.block.SPPF,
    torch.nn.modules.pooling.MaxPool2d,
    torch.nn.modules.upsampling.Upsample,
    ultralytics.nn.modules.conv.Concat,
    ultralytics.nn.modules.head.Detect,
    ultralytics.nn.modules.block.DFL
])

torch.load("/content/best.pt", weights_only=False)
model = YOLO("/content/best.pt", task="detect")

YOLO MODEL LOADING

In [21]:
from ultralytics import YOLO

model = YOLO("/content/best.pt", task="detect")
print("YOLO model loaded successfully")


YOLO model loaded successfully


VIT DATASET

In [49]:
import os, shutil, random

base = "/content/vit_lamp_dataset"
splits = ["train", "val", "test"]
classes = ["ON", "OFF"]

# Create folders
for s in splits:
    for c in classes:
        os.makedirs(f"{base}/{s}/{c}", exist_ok=True)

# Manually list images (adjust if filenames differ)
ON_images = [
    "/content/vit_lamp_dataset/online-puja.jpg",
    "/content/vit_lamp_dataset/SriMalaiperumalIdol.jpg",
    "/content/vit_lamp_dataset/images (4).jpg",
    "/content/vit_lamp_dataset/images (3).jpg",
    "/content/vit_lamp_dataset/images (2).jpg",
    "/content/vit_lamp_dataset/images (1).jpg",
    "/content/vit_lamp_dataset/4d9df308f558b1a04d6e1b55714ad4b3.jpg",
    "/content/vit_lamp_dataset/1abd640fc2550ea8c3d563744393aa5b.jpg",
    "/content/vit_lamp_dataset/WhatsApp Image 2026-01-14 at 1.05.11 PM.jpeg",
    "/content/vit_lamp_dataset/WhatsApp Image 2026-01-14 at 1.05.12 PM.jpeg",
    "/content/vit_lamp_dataset/How_to_Worship_Tirupati_Balaji_Idol_at_Home_Easily_7165b689-866a-4a8e-ae0d-0bc7a086fbec.webp"
]

OFF_images = [
    "/content/vit_lamp_dataset/images (5).jpg",
    "/content/vit_lamp_dataset/61nybWUdHML._AC_UF350,350_QL80_.jpg"
]

random.shuffle(ON_images)
random.shuffle(OFF_images)

def split_copy(images, label):
    train = images[:int(0.6*len(images))]
    val   = images[int(0.6*len(images)):int(0.8*len(images))]
    test  = images[int(0.8*len(images)):]

    for img in train:
        shutil.copy(img, f"{base}/train/{label}/")
    for img in val:
        shutil.copy(img, f"{base}/val/{label}/")
    for img in test:
        shutil.copy(img, f"{base}/test/{label}/")

split_copy(ON_images, "ON")
split_copy(OFF_images, "OFF")

print("ViT dataset created successfully")


ViT dataset created successfully


In [50]:
!find vit_lamp_dataset -type f | wc -l
!find vit_lamp_dataset/train -type f


38
vit_lamp_dataset/train/OFF/images (5).jpg
vit_lamp_dataset/train/ON/How_to_Worship_Tirupati_Balaji_Idol_at_Home_Easily_7165b689-866a-4a8e-ae0d-0bc7a086fbec.webp
vit_lamp_dataset/train/ON/1abd640fc2550ea8c3d563744393aa5b.jpg
vit_lamp_dataset/train/ON/4d9df308f558b1a04d6e1b55714ad4b3.jpg
vit_lamp_dataset/train/ON/WhatsApp Image 2026-01-14 at 1.05.12 PM.jpeg
vit_lamp_dataset/train/ON/images (3).jpg
vit_lamp_dataset/train/ON/images (2).jpg
vit_lamp_dataset/train/ON/images (1).jpg
vit_lamp_dataset/train/ON/online-puja.jpg
vit_lamp_dataset/train/ON/images (4).jpg
vit_lamp_dataset/train/ON/SriMalaiperumalIdol.jpg


In [51]:
!find vit_lamp_dataset -type d


vit_lamp_dataset
vit_lamp_dataset/val
vit_lamp_dataset/val/OFF
vit_lamp_dataset/val/ON
vit_lamp_dataset/train
vit_lamp_dataset/train/OFF
vit_lamp_dataset/train/ON
vit_lamp_dataset/test
vit_lamp_dataset/test/OFF
vit_lamp_dataset/test/ON


In [31]:
!pip install transformers timm accelerate scikit-learn


In [52]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from transformers import ViTForImageClassification, ViTImageProcessor
from sklearn.metrics import accuracy_score


VIT IMAGE TRANSFORM

In [53]:
processor = ViTImageProcessor.from_pretrained(
    "google/vit-base-patch16-224"
)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])


CLASS MAPPING

In [54]:
train_ds = datasets.ImageFolder(
    "vit_lamp_dataset/train",
    transform=transform
)
val_ds = datasets.ImageFolder(
    "vit_lamp_dataset/val",
    transform=transform
)
test_ds = datasets.ImageFolder(
    "vit_lamp_dataset/test",
    transform=transform
)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=4)
test_loader = DataLoader(test_ds, batch_size=4)

print("Class mapping:", train_ds.class_to_idx)


Class mapping: {'OFF': 0, 'ON': 1}


VIT IMAGE CLASSIFICATION

In [55]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
for param in model.vit.parameters():
    param.requires_grad = False


In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.classifier.parameters(),
    lr=1e-3
)


TRAIN LOOP

In [58]:
EPOCHS = 8

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=imgs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation
    model.eval()
    preds, gts = [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            outputs = model(pixel_values=imgs)
            preds.extend(outputs.logits.argmax(dim=1).cpu().tolist())
            gts.extend(labels.tolist())

    acc = accuracy_score(gts, preds)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Loss: {total_loss:.3f} | "
        f"Val Acc: {acc:.3f}"
    )


Epoch 1/8 | Train Loss: 1.985 | Val Acc: 0.857
Epoch 2/8 | Train Loss: 0.684 | Val Acc: 0.857
Epoch 3/8 | Train Loss: 0.665 | Val Acc: 0.857
Epoch 4/8 | Train Loss: 0.633 | Val Acc: 0.857
Epoch 5/8 | Train Loss: 0.516 | Val Acc: 0.857
Epoch 6/8 | Train Loss: 0.413 | Val Acc: 0.857
Epoch 7/8 | Train Loss: 0.259 | Val Acc: 1.000
Epoch 8/8 | Train Loss: 0.111 | Val Acc: 1.000


SAVING VIT MODEL

In [59]:
model.save_pretrained("/content/vit_lamp_model")
processor.save_pretrained("/content/vit_lamp_model")

print("ViT model saved")


ViT model saved


INFERENCE

In [61]:
from PIL import Image

img = Image.open("vit_lamp_dataset/test/ON/" +
                 os.listdir("vit_lamp_dataset/test/ON")[0])

img_t = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    out = model(pixel_values=img_t)
    probs = torch.softmax(out.logits, dim=1)

print({
    "OFF": float(probs[0][0]),
    "ON": float(probs[0][1])
})


{'OFF': 0.03009096346795559, 'ON': 0.9699090123176575}


SIAMESE CNN — FULL TRAINING CODE (POC)

In [62]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import random


IMAGE TRANSFORMS

In [63]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


DEFINE IMAGE PAIRS

In [79]:
pairs = [
    ("SriMalaiperumalIdol.jpg", "SriMalaiperumalIdol.jpg", 1),
    ("SriMalaiperumalIdol.jpg", "online-puja.jpg", 0),
    ("images (4).jpg", "images (4).jpg", 1),
    ("images (5).jpg", "106031861.avif", 0),
    ("images (3).jpg", "images (3).jpg", 1),
    ("images (2).jpg", "Screenshot 2026-01-13 163738.png", 0),
]


DATASET CLASS

In [70]:
class SiameseDataset(torch.utils.data.Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1, img2, label = self.pairs[idx]

        img1 = transform(Image.open(img1).convert("RGB"))
        img2 = transform(Image.open(img2).convert("RGB"))

        return img1, img2, torch.tensor(label, dtype=torch.float32)


SIAMESE MODEL

In [71]:
class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = models.resnet18(pretrained=True)
        self.encoder = nn.Sequential(*list(backbone.children())[:-1])
        self.fc = nn.Linear(512, 128)

    def forward_once(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)


TRAIN SETUP

In [72]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = SiameseNet().to(device)
criterion = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

dataset = SiameseDataset(pairs)
loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)


TRAIN LOOP

In [74]:
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    for img1, img2, label in loader:
        img1, img2 = img1.to(device), img2.to(device)
        label = label.to(device)
        label = label * 2 - 1  # Convert 0/1 → -1/1

        optimizer.zero_grad()
        out1, out2 = model(img1, img2)
        loss = criterion(out1, out2, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss:.4f}")


Epoch 1/10 | Loss: 1.0024
Epoch 2/10 | Loss: 0.7420
Epoch 3/10 | Loss: 0.1405
Epoch 4/10 | Loss: 0.1297
Epoch 5/10 | Loss: 0.0587
Epoch 6/10 | Loss: 0.0710
Epoch 7/10 | Loss: 0.0485
Epoch 8/10 | Loss: 0.3115
Epoch 9/10 | Loss: 0.0282
Epoch 10/10 | Loss: 0.0606


INFERENCE

In [80]:
import torch.nn.functional as F

def similarity(imgA, imgB):
    imgA = transform(Image.open(imgA).convert("RGB")).unsqueeze(0).to(device)
    imgB = transform(Image.open(imgB).convert("RGB")).unsqueeze(0).to(device)

    with torch.no_grad():
        embA, embB = model(imgA, imgB)
        score = F.cosine_similarity(embA, embB).item()

    return round(score, 3)

print(similarity("SriMalaiperumalIdol.jpg", "SriMalaiperumalIdol.jpg"))
print(similarity("images (3).jpg", "Screenshot 2026-01-13 163738.png"))


1.0
0.999


DARVE-CORE — FINAL INTEGRATION

YOLO + ViT + Siamese → One Decision

1. darve_yolo.py — Object Presence

In [93]:
%%writefile darve_yolo.py
import torch
from ultralytics import YOLO
import ultralytics.nn.modules.conv
import ultralytics.nn.modules.block
import ultralytics.nn.modules.head

# 🔒 Allow Ultralytics classes for PyTorch 2.6+
torch.serialization.add_safe_globals([
    torch.nn.modules.container.Sequential,
    ultralytics.nn.modules.conv.Conv,
    torch.nn.modules.conv.Conv2d,
    torch.nn.modules.batchnorm.BatchNorm2d,
    torch.nn.modules.activation.SiLU,
    ultralytics.nn.modules.block.C2f,
    torch.nn.modules.container.ModuleList,
    ultralytics.nn.modules.block.Bottleneck,
    ultralytics.nn.modules.block.SPPF,
    torch.nn.modules.pooling.MaxPool2d,
    torch.nn.modules.upsampling.Upsample,
    ultralytics.nn.modules.conv.Concat,
    ultralytics.nn.modules.head.Detect,
    ultralytics.nn.modules.block.DFL,
    ultralytics.nn.tasks.DetectionModel,
])

# Load YOLO safely
yolo = YOLO("/content/best.pt")

def get_object_score(image_path: str) -> float:
    results = yolo(image_path, conf=0.3)
    boxes = results[0].boxes

    if boxes is None or len(boxes) == 0:
        return 0.0

    scores = [float(b.conf) for b in boxes]
    return round(sum(scores) / len(scores), 3)



Overwriting darve_yolo.py


2. darve_vit.py — Lamp ON / OFF

In [87]:
%%writefile darve_vit.py
import torch
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTForImageClassification.from_pretrained(
    "/content/vit_lamp_model"
).to(device)
model.eval()

processor = ViTImageProcessor.from_pretrained(
    "/content/vit_lamp_model"
)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])

def get_ritual_score(image_path: str) -> float:
    img = Image.open(image_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(pixel_values=img).logits
        probs = torch.softmax(logits, dim=1)

    # Index 1 = ON
    return round(float(probs[0][1]), 3)


Writing darve_vit.py


3. darve_siamese.py — Scene Consistency

In [88]:
%%writefile darve_siamese.py
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

class SiameseNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        backbone = models.resnet18(pretrained=True)
        self.encoder = torch.nn.Sequential(*list(backbone.children())[:-1])
        self.fc = torch.nn.Linear(512, 128)

    def forward_once(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

    def forward(self, x1, x2):
        return self.forward_once(x1), self.forward_once(x2)

model = SiameseNet().to(device)
model.eval()  # PoC model

def get_scene_score(img_today: str, img_yesterday: str) -> float:
    img1 = transform(Image.open(img_today).convert("RGB")).unsqueeze(0).to(device)
    img2 = transform(Image.open(img_yesterday).convert("RGB")).unsqueeze(0).to(device)

    with torch.no_grad():
        e1, e2 = model(img1, img2)
        score = F.cosine_similarity(e1, e2).item()

    return round(score, 3)


Writing darve_siamese.py


4. darve_core.py — THE BRAIN

In [96]:
%%writefile darve_core.py
from dataclasses import dataclass
from typing import List

@dataclass
class DarveSignals:
    object_score: float
    ritual_score: float
    scene_score: float

@dataclass
class DarveResult:
    status: str
    confidence: float
    reasons: List[str]

def darve_decide(s: DarveSignals) -> DarveResult:
    reasons = []

    if s.object_score < 0.5:
        reasons.append("Ritual objects missing")

    if s.ritual_score < 0.6:
        reasons.append("Lamp not lit")

    if s.scene_score < 0.7:
        reasons.append("Different shrine detected")

    confidence = round(
        0.4 * s.object_score +
        0.4 * s.ritual_score +
        0.2 * s.scene_score,
        3
    )

    if confidence >= 0.80:
        status = "POOJA_DONE"
    elif confidence >= 0.6:
        status = "UNCLEAR"
    else:
        status = "POOJA_NOT_DONE"

    return DarveResult(
        status=status,
        confidence=confidence,
        reasons=reasons if reasons else ["All checks passed"]
    )


Overwriting darve_core.py


5. test_darve.py — END-TO-END TEST

In [91]:
%%writefile test_darve.py
from darve_yolo import get_object_score
from darve_vit import get_ritual_score
from darve_siamese import get_scene_score
from darve_core import DarveSignals, darve_decide

img_today = "SriMalaiperumalIdol.jpg"
img_yesterday = "SriMalaiperumalIdol.jpg"

signals = DarveSignals(
    object_score=get_object_score(img_today),
    ritual_score=get_ritual_score(img_today),
    scene_score=get_scene_score(img_today, img_yesterday)
)

result = darve_decide(signals)
print(result)


Overwriting test_darve.py


In [97]:
!python test_darve.py


2026-01-14 08:41:32.302699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768380092.328132   44895 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768380092.336071   44895 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768380092.359239   44895 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768380092.359288   44895 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768380092.359293   44895 computation_placer.cc:177] computation placer alr

In [98]:
import subprocess
import pandas as pd
import re

# Run the test_darve.py script and capture its output
result = subprocess.run(['python', 'test_darve.py'], capture_output=True, text=True)
output_str = result.stdout.strip()

# Parse the output string
# Example output: DarveResult(status='POOJA_DONE', confidence=0.841, reasons=['All checks passed'])
status_match = re.search(r"status='([^']+)'", output_str)
confidence_match = re.search(r"confidence=([\d.]+)", output_str)
reasons_match = re.search(r"reasons=\[([^]]*)\]", output_str)

status = status_match.group(1) if status_match else 'N/A'
confidence = float(confidence_match.group(1)) if confidence_match else 'N/A'
reasons_str = reasons_match.group(1) if reasons_match else ''
reasons = [r.strip("'") for r in reasons_str.split(',')] if reasons_str else []

# Create a pandas DataFrame
df_result = pd.DataFrame({
    'Metric': ['Status', 'Confidence', 'Reasons'],
    'Value': [status, confidence, ', '.join(reasons)]
})

# Display the DataFrame
display(df_result)

,Metric,Value
0,Status,POOJA_DONE
1,Confidence,0.836
2,Reasons,All checks passed
